# **CÂU HỎI**

![alt text](https://www.ctdb.hcmus.edu.vn/wp-content/uploads/2017/04/17880008_1006985519432248_7695519039665360507_o-1.jpg)

Chắc hẳn ai trong chúng ta cũng từng gặp một tấm poster như thế này trong đời. Ít nhất thì đây là lần đầu tiên nếu trước đó bạn chưa thấy.

Có thể vì tính thẩm mỹ, tiết kiệm không gian hoặc vì một lý do nào đó không biết được, chúng ta chỉ thấy được các logo của công ty trên tấm poster này.

Vậy, bạn có thể kể hết tên các công ty có mặt trên poster này không. Tôi đoán đa phần là không, ít nhất thì tôi không biết.

Xuất phát từ nhu cầu muốn biết công ty nào được in trên các poster của chính chúng tôi và giúp nhưng người có nhu cầu giống như thế, chúng tôi muốn phát triển một hệ thống có thể nhận biết được công ty thông qua logo của họ.

Do thời gian có hạn và tài nguyên không đủ, chúng tôi chỉ làm một ví dụ với 5 công ty là vng, viettel, fpt, kms, fujinet. Chúng tôi sẽ phát triển thêm trong tương lại khi gọi vốn thành công từ các sharks :)). Dữ liệu của chúng tôi được thu thập từ internet. Cụ thể là từ google images. Chúng tôi chỉ lấy những tấm ảnh được gán nhán là được phép sử dụng (free).

Cuối cùng, nếu bạn thích ý tưỡng này, hãy đầu tư cho chúng tôi. Xin cảm ơn.

# **UPDATE: THU THẬP DỮ LIỆU**

Do dữ liệu thu thập từ google search không tốt như dự đoán, nhóm sẽ sử dụng cách khác để thu thập dữ liệu.

Data bao gồm các hình ảnh được lấy từ trang web [Brands Vietnam](https://www.brandsvietnam.com/). Trang web này bao gồm các hình ảnh, thông tin và các sự kiện xảy ra trong các năm gần đây của các công ty Việt Nam.

Nhóm đã tiến hành kiểm tra và xác nhận là các hình ảnh trên trang này được phép thu thập. Cụ thể, nhóm sẽ tìm các công ty trong trang [danh bạ công ty](https://www.brandsvietnam.com/danhba/2-Congty). Với mỗi công ty, nhóm sẽ vào tất cả các bài đăng và thu thập tất cả các ảnh trong mỗi bài đăng. Sau đó nhóm sẽ tiến hành loại bỏ các ảnh xấu, tiền xử lý dữ liệu hoặc có thể sử dụng các biện pháp tăng cường dữ liệu nếu dữ liệu thu thập được quá ít.

Để tiện cho việc thu thập dữ liệu, 5 công ty nhóm sẽ thu thập là vng, viettel, fpt, tiki và thế giới di động. Thay vì vng, viettel, fpt, kms, fujinet như ban đầu.

# **CODE**

In [0]:
#Import library

from requests_html import HTMLSession
import requests
import os
import argparse
import sys
import json
import time

In [0]:
# Define url of companies
urls = {
    'vinamilk': 'https://www.brandsvietnam.com/keyword/54-Vinamilk',
    'fpt': 'https://www.brandsvietnam.com/keyword/398-FPT',
    'tiki': 'https://www.brandsvietnam.com/keyword/1114-Tiki',
    'viettel': 'https://www.brandsvietnam.com/keyword/200-Viettel',
    'tgdd': 'https://www.brandsvietnam.com/keyword/613-The-Gioi-Di-Dong',
}

In [0]:
# Create html session

session = HTMLSession()

In [0]:
# Function: Collect image of one post
# Return: New index

def collect_images_from_post(post_url, output_dir, start_index=0, time_out=10):
  imgs = []
  try:
    res = session.get(post_url)
    post_content = res.html.find('.reading-content', first=True)
    imgs = post_content.find('img')
  except:
    print("Unexpected error: ", sys.exc_info())
    return start_index

  index = start_index
  for img in imgs:
    img_url = img.attrs['src']
    ext = img_url.split('.')[-1]
    try:
      raw_img = requests.get(img_url, timeout=time_out).content
      f = open(os.path.join(output_dir, f'img_{index}.{ext}'), 'wb')
      f.write(raw_img)
      f.close
      index += 1
    except:
      print("Unexpected error: ", sys.exc_info())
  
  return index

In [0]:
# Function: Collect image of one company
# Return: New index

def collect_images_of_company(company_url, output_dir, start_index=0, time_out=10):
  res = session.get(company_url)
  content = res.html.find('.comment-list', first=True)
  posts = content.find('.media-body>a')

  index = start_index
  for post in posts:
    post_url = post.attrs['href']
    index = collect_images_from_post(post_url=post_url, output_dir=output_dir, start_index=index, time_out=time_out)
  
  return index

In [0]:
# Function: Collect all images

def collect_all_images(urls, output_dir='.'):
  # Create output directory if not exist
  if not os.path.exists(output_dir):
    os.mkdir(output_dir)
  
  for company, company_url in urls.items():
    company_dir = os.path.join(output_dir, company)
    if not os.path.exists(company_dir):
      os.mkdir(company_dir)
    
    # Collect images
    collect_images_of_company(company_url=company_url, output_dir=company_dir)

In [0]:
# Collect images

collect_all_images(urls, './dataset/images')